# Introduction to GenePattern Notebook

This document should help you understand how to run an analysis in the GenePattern Notebook environment. In it you will perform a simple preprocessing step and then view the results in a heat map. 

**Instructions are given in blue boxes, such as with the one below.**

<div class="alert alert-info">Sign in to GenePattern by clicking the login button or entering your username and password into the form below.</div>

## Dataset information
In this example we will preprocess a dataset of 38 samples of leukemia, 27 of subtype ALL and 11 of subtype AML. The data was created on a microarray platform, but the resulting [GCT](http://software.broadinstitute.org/cancer/software/genepattern/file-formats-guide#GCT) file is compatible with RNA-Seq, as well as any other data type that can be expressed with samples as columns and features as rows.

In [1]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.GPAuthWidget(genepattern.register_session("https://gp-beta-ami.genepattern.org/gp", "", ""))

GPAuthWidget()

## Step 1: Remove non-varying (noise) genes from dataset (PreprocessDataset)

Preprocess gene expression data to remove platform noise and genes that have little variation. You can test this step by starting a job using parameters entered into the form below.

### Considerations

* PreprocessDataset can preprocess the data in one or more ways (in this order):
    1. Set threshold and ceiling values. Any value lower/higher than the threshold/ceiling value is reset to the threshold/ceiling value.
    2. Convert each expression value to the log base 2 of the value.
    3. Remove genes (rows) if a given number of its sample values are less than a given threshold.
    4. Remove genes (rows) that do not have a minimum fold change or expression variation.
    5. Discretize or normalize the data.
* If you did not generate the expression data, check whether preprocessing steps have already been taken before running the PreprocessDataset module.
* Learn more by reading about the [PreprocessDataset](https://genepattern.broadinstitute.org/gp/getTaskDoc.jsp?name=PreprocessDataset) module.

<div class="alert alert-info">
- Change the *min fold change* parameter to 10
- Execute the cell by doing **one** of the following:
    - Type `Shift-Enter`
    - Choose the Cell &#x2192; Run Cells menu option
    - <img align="left" src="https://datasets.genepattern.org/images/jupyter-run.png"> &#x2190; Click the Run icon  on the navigation bar under the menu.

In [2]:
preprocessdataset_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00020')
preprocessdataset_job_spec = preprocessdataset_task.make_job_spec()
preprocessdataset_job_spec.set_parameter("input.filename", "https://datasets.genepattern.org/data/all_aml/all_aml_train.gct")
preprocessdataset_job_spec.set_parameter("threshold.and.filter", "1")
preprocessdataset_job_spec.set_parameter("floor", "100")
preprocessdataset_job_spec.set_parameter("ceiling", "20000")
preprocessdataset_job_spec.set_parameter("min.fold.change", "3")
preprocessdataset_job_spec.set_parameter("min.delta", "100")
preprocessdataset_job_spec.set_parameter("num.outliers.to.exclude", "0")
preprocessdataset_job_spec.set_parameter("row.normalization", "0")
preprocessdataset_job_spec.set_parameter("row.sampling.rate", "1")
preprocessdataset_job_spec.set_parameter("threshold.for.removing.rows", "")
preprocessdataset_job_spec.set_parameter("number.of.columns.above.threshold", "")
preprocessdataset_job_spec.set_parameter("log2.transform", "0")
preprocessdataset_job_spec.set_parameter("output.file.format", "3")
preprocessdataset_job_spec.set_parameter("output.file", "<input.filename_basename>.preprocessed")
genepattern.GPTaskWidget(preprocessdataset_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00020')

## Step 2: View results in heat map format (HeatMapViewer)

Display a heat map of the preprocessed gene expression data. Since the *min fold change* was stringent in the previous step, this will show only the genes that vary highly in expression.

### Considerations

* HeatMapViewer displays gene expression data as a heat map, which makes it easier to see patterns in the numeric data. Gene names are row labels and sample names are column labels.
* Notebooks may contain any number of visualizers.
* The features displayed here use Ensembl IDs. In the next section we will convert these IDs to gene names.
* Learn more by reading about the [HeatMapViewer](https://genepattern.broadinstitute.org/gp/getTaskDoc.jsp?name=HeatMapViewer) module.

### Note on instructions
- Many result files have similar names and differ only in their suffixes. When we indicate a file as `<filename>.gct` for example, we mean the result file that has the extension `.gct`.

<div class="alert alert-info">
- When the job above shows that it is completed (status in the upper right corner of the job cell displays **Completed**):
- Click the link for the `<filename>.preprocessed.gct` result file.
- You will see a list of choices.
- Select **Send to Existing GenePattern Cell**.
- You will see a list of available cells.
- Select the **HeatMapViewer** cell.
- You will see the file populated in the *dataset* parameter of the **HeatMapViewer** cell below.
- Launch the **HeatMapViewer** job by clicking **Run**.

In [3]:
heatmapviewer_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00010')
heatmapviewer_job_spec = heatmapviewer_task.make_job_spec()
heatmapviewer_job_spec.set_parameter("dataset", "")
genepattern.GPTaskWidget(heatmapviewer_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00010')

## Step 3: Convert feature identifiers to gene names (CollapseDataset)

You can see that the dataset that was created uses an identifier system from [Ensembl](https://uswest.ensembl.org/index.html) to name each gene (really transcript). We would like to convert these to more easily understandable gene names. The standard identification system used for genes is the [HUGO (HUman Genome Organization)](https://www.genenames.org/) standard. 

<div class="alert alert-info">
- In the cell below, for the *dataset file* parameter, click the dropdown arrow on the right side of the input box.
- You will see all of the available result files in this notebook that can be sent to this input.
- Select `<filename>.preprocessed.gct`
- Click **Run**.

In [4]:
collapsedataset_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00134')
collapsedataset_job_spec = collapsedataset_task.make_job_spec()
collapsedataset_job_spec.set_parameter("dataset.file", "")
collapsedataset_job_spec.set_parameter("chip.platform", "ftp://ftp.broadinstitute.org/pub/gsea/annotations/HU6800.chip")
collapsedataset_job_spec.set_parameter("collapse.mode", "Maximum")
collapsedataset_job_spec.set_parameter("output.file.name", "<dataset.file_basename>.collapsed")
genepattern.GPTaskWidget(collapsedataset_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00134')